# Keras Image Classification for Small Data

This notebook uses the Keras framework to train an image classifier using a small dataset.

Source: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

## Packages

In [1]:
import os # for data prep
import shutil # for data prep
import re #for data prep

The lastest version of Keras does somehow not work on GCP Datalab. 
This configuration works:

In [ ]:
!pip install click==6.7
!pip install Flask==0.12
!pip install Cython==0.25.2
!pip install funcsigs==1.0.2
!pip install h5py==2.6.0
!pip install itsdangerous==0.24
!pip install Jinja2==2.9.4
!pip install Keras==2.2.0
!pip install MarkupSafe==0.23
!pip install mock==2.0.0
!pip install numpy==1.12.0
!pip install olefile==0.44
!pip install pbr==1.10.0
!pip install Pillow==4.0.0
!pip install protobuf==3.1.0.post1
!pip install PyYAML==3.12
!pip install scipy==0.18.1
!pip install six==1.10.0
!pip install tensorflow==1.10
!pip install Theano==0.8.2
!pip install Werkzeug==0.11.15
!pip install gunicorn==19.6.0

In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

Using TensorFlow backend.


## Data Preparation

The data comes in the followinf structure:

data/ <br />
--train/<br />
--- dogs/<br />
----- dog.001.jpg<br />
----- dog.002.jpg<br />
----- ...<br />
--- cats/<br />
----- cat.001.jpg<br />
----- cat.002.jpg<br />
----- ...<br />
-- validation/<br />
--- dogs/<br />
----- dog.001.jpg<br />
----- dog.002.jpg<br />
----- ...<br />
--- cats/<br />
----- cat.001.jpg<br />
----- cat.002.jpg<br />
----- ...<br />

We have to load it from a GCP bucket.

In [3]:
bucket_path = "gs://small-image-classifier"

In [4]:
gcs list --objects $bucket_path

Name,Type,Size,Updated
data/train/cats/cat.0.jpg,image/jpeg,12414,2019-02-21 14:58:54.001000+00:00
data/train/cats/cat.1.jpg,image/jpeg,16880,2019-02-21 14:59:23.942000+00:00
data/train/cats/cat.10.jpg,image/jpeg,34315,2019-02-21 15:10:18.639000+00:00
data/train/cats/cat.100.jpg,image/jpeg,28377,2019-02-21 15:02:37.924000+00:00
data/train/cats/cat.101.jpg,image/jpeg,11291,2019-02-21 15:02:27.021000+00:00
data/train/cats/cat.102.jpg,image/jpeg,28635,2019-02-21 15:01:48.967000+00:00
data/train/cats/cat.103.jpg,image/jpeg,24853,2019-02-21 15:01:57.138000+00:00
data/train/cats/cat.104.jpg,image/jpeg,18941,2019-02-21 15:00:27.021000+00:00
data/train/cats/cat.105.jpg,image/jpeg,6159,2019-02-21 15:00:30.909000+00:00
data/train/cats/cat.106.jpg,image/jpeg,20232,2019-02-21 15:01:11.851000+00:00


In [5]:
from PIL import Image
import io

# Define the bucket and and an example image to read
sample_img_path = bucket_path + "/data/train/cats/cat.0.jpg"

# List all the objects in your bucket, and read the example image file
%gcs read --object $sample_img_path --variable sample_img

#Convert byte to image file
sample_img = Image.open(io.BytesIO(sample_img))

# Print the image content (see it is in PNG format) and show it
print(type(sample_img))


<class 'PIL.JpegImagePlugin.JpegImageFile'>


Ok, this works. For the flow_from_directory function to work we need the data on the VM anyway:

In [61]:
if  not os.path.exists("./small-image-classifier"):
  os.system('gsutil cp -r gs://small-image-classifier/ .')
  

In [62]:
os.listdir("./small-image-classifier/data/train/")

['cats', 'dogs']

## Data pre-processing and data augmentation

Apply random transformations, so that the model would never see the exact same picture twice. This helps prevent overfitting and helps the model generalize better.

See preview how it works:



In [65]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = sample_img  # this is a PIL image

x = img_to_array(sample_img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `/preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

Make sure Input Shapes are formatted correctly

In [64]:
img_width, img_height = 150, 150


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
print(input_shape)

(150, 150, 3)


In [63]:
train_data_dir = './small-image-classifier/data/train'
validation_data_dir = './small-image-classifier/data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

Below is our first model, a simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers. 

In [66]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)


On top of it we stick two fully-connected layers. We end the model with a single unit and a sigmoid activation, which is perfect for a binary classification. To go with it we will also use the binary_crossentropy loss to train our model.

In [67]:
model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


Let's prepare our data. We will use .flow_from_directory() to generate batches of image data (and their labels) directly from our jpgs in their respective folders.

In [68]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


We can now use these generators to train our model. 

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
model_save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50
125/125 [==============================] - 76s 610ms/step - loss: 0.7254 - acc: 0.5130 - val_loss: 0.6844 - val_acc: 0.6300
Epoch 2/50
125/125 [==============================] - 76s 604ms/step - loss: 0.6845 - acc: 0.5985 - val_loss: 0.7154 - val_acc: 0.5500
Epoch 3/50
125/125 [==============================] - 75s 602ms/step - loss: 0.6471 - acc: 0.6535 - val_loss: 0.6502 - val_acc: 0.6275
Epoch 4/50
125/125 [==============================] - 75s 596ms/step - loss: 0.6256 - acc: 0.6630 - val_loss: 0.5805 - val_acc: 0.6975
Epoch 5/50
125/125 [==============================] - 75s 601ms/step - loss: 0.6042 - acc: 0.6855 - val_loss: 0.5817 - val_acc: 0.6863
Epoch 6/50
125/125 [==============================] - 75s 602ms/step - loss: 0.5743 - acc: 0.7075 - val_loss: 0.5452 - val_acc: 0.7288
Epoch 7/50
125/125 [==============================] - 75s 602ms/step - loss: 0.5651 - acc: 0.7180 - val_loss: 0.5676 - val_acc: 0.6963
Epoch 8/50
125/125 [==============================] - 7